Implement the demand response

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt

Function collection

In [3]:
def extract_cyc(profile) -> tuple:
    """
    Extract the operation cycles from the profile.
    Returns the start times and profile of the cycles.
    """
    cycles = [] # list of operation cycles
    cycle = [] # current operation cycle
    for i in range(len(profile)):
        if profile[i] > 0:
            cycle.append(i)
        else: # if the profile is 0, it means the cycle is over
            if cycle:
                cycles.append(cycle)
                cycle = []
    if cycle:
        cycles.append(cycle)
        
    return [c[0] for c in cycles], [profile[c] for c in cycles]

def constrain_nonflex(model, cycle_start, cycle_profile, duration, tolerance, name: str):
    """
    Build the optimization constraints for the non-flexible appliances.
    """
    # Create variables
    num_cycle = len(cycle_start)

    app_energy = model.addVars(num_cycle, duration, vtype=gp.GRB.CONTINUOUS, lb=0, name=name+"_power")
    app_start = model.addVars(num_cycle, duration, vtype=gp.GRB.BINARY, name=name+"_start")

    for i, start in enumerate(cycle_start):
        len_cycle = len(cycle_profile[i])
        # constraint (1)&(2): The starting time of each operation should not deviate from the consumer preference
        # time by more than a certain time tolerance. I think it doesn't make any sense to bring the operation ahead
        # of the preference time, so we only consider the delay.
        model.addConstr(gp.quicksum(app_start[i, j] for j in range(start, start + tolerance[i] + 1)) == 1, name=name+f"_start_constr_{i}")

        # constraint: all the operations should complete before the end of the day
        model.addConstr(gp.quicksum(app_start[i, j] for j in range(start, duration-len_cycle+1)) == 1, name=name+f"end_constr_{i}")

        # constraint: every operation should only start once
        model.addConstr(gp.quicksum(app_start[i, j] for j in range(duration)) == 1, name=name+f"_unique_constr_{i}")

        # constraint (3): the operation should not start before the end of the previous operation
        if i > 0:
            model.addConstr(gp.quicksum((app_start[i, j]-app_start[i-1, j])*j for j in range(start, start + tolerance[i] + 1)) >= len_cycle, name=name+f"_overlap_constr_{i}")
        # constraint (4): the shifted operation is calculated as the convolution of the profile and the decision variable
        for j in range(duration):
            conv = gp.quicksum(app_start[i, j-k] * cycle_profile[i][k] for k in range(len_cycle) if j-k >= 0)
            model.addConstr(app_energy[i, j] == conv, name=f"wm_power_constr_{i}_{j}")

    return app_energy, app_start

def constrain_flex(model, soc_init, soc_demand, soc_max, pos_rate, neg_rate, eff, t_in, t_out, duration, name: str):
    """
    Build the optimization constraints for the flexible appliances.
    """
    # constraint (6): the power rate is limited by the maximum charging and discharging power rate
    app_pos_energy = model.addVars(duration, vtype=gp.GRB.CONTINUOUS, lb=0, ub=pos_rate, name=name+"_pos_power")
    app_neg_energy = model.addVars(duration, vtype=gp.GRB.CONTINUOUS, lb=neg_rate, ub=0, name=name+"_neg_power")

    # assume the lower bound of soc is internally higher than 0 
    app_soc = model.addVars(t_out-t_in+1, vtype=gp.GRB.CONTINUOUS, lb=0, ub=soc_max, name=name+"_soc")

    # constraint (5): the energy demand has to be satisfied within the time window [plug_in, plug_out]
    model.addConstr(app_soc[t_out-t_in] - soc_init == soc_demand, name=name+"_demand_constr")
    # constraint: the initial state of charge is set to the initial value
    model.addConstr(app_soc[0] == soc_init, name=name+f"_soc_init_constr")

    for t in range(duration):
        if t < t_in or t >= t_out:
            # constraint (7): the power rate is limited to 0 outside the time window [plug_in, plug_out]
            model.addConstr(app_pos_energy[t] == 0, name=name+f"_pos_power_constr_{t}")
            model.addConstr(app_neg_energy[t] == 0, name=name+f"_neg_power_constr_{t}")
        
        # transition function of the state of charge
        else:
            model.addConstr(app_soc[t-t_in+1] == app_soc[t-t_in] + app_pos_energy[t]*eff + app_neg_energy[t]/eff, name=name+f"_soc_pos_constr_{t}") 
            model.addConstr(app_pos_energy[t] * app_neg_energy[t] == 0, name=name+f"_pos_neg_constr_{t}") # only one of the two can be non-zero
    
    return app_pos_energy, app_neg_energy, app_soc 

def constrain_pv(model, battery_init, battery_max, energy_rate, eff, duration, name: str):
    """
    Build the optimization constraints for the PV system.
    """
    battery_pos_energy = model.addVars(duration, vtype=gp.GRB.CONTINUOUS, lb=0, ub=energy_rate, name=name+"_pos_battery_charge") 
    battery_neg_energy = model.addVars(duration, vtype=gp.GRB.CONTINUOUS, lb=-energy_rate, ub=0, name=name+"_neg_battery_charge") 
    battery_soc = model.addVars(duration+1, vtype=gp.GRB.CONTINUOUS, lb=0, ub=battery_max, name=name+"battery_soc")

    # initialize the battery soc
    model.addConstr(battery_soc[0] == battery_init, name=name+"_battery_soc_init")
    # model.addConstr(battery_soc[duration] == battery_init, name=name+"_battery_soc_end")
    for t in range(duration):
        # transition function of soc
        model.addConstr(battery_soc[t+1] == battery_pos_energy[t]*eff + battery_neg_energy[t]/eff + battery_soc[t], name=name+f"_battery_soc_constr_{t}")

        # constraint: the aggregation of total demand and energy charged into the battery should be greater than pv
        # model.addConstr(pv_pred[t] - demand[t] <= battery_pos_energy[t] + battery_neg_energy[t], name=name+f"_battery_pv_{t}")
        # constraint: at each time step, the battery can only charge or discharge
        model.addConstr(battery_pos_energy[t] * battery_neg_energy[t] == 0, name=name+f"_battery_pos_neg_constr_{t}")

    return battery_pos_energy, battery_neg_energy, battery_soc

def get_profile(var, row, col, type='nonflex'):
    """
    Get the profile of a gurobi variable.
    row: number of cycles
    col: number of time steps
    """
    if type == 'nonflex':
        return [sum(var[i, t].X for i in range(row)) for t in range(col)]
    
    if type == 'flex':
        return [var[t].X for t in range(col)]

def get_tolerance(cyc_start, cyc_profile, duration, max_delay=24):
    """
    Get the tolerance for the operation cycles.
    The tolerance is the maximum delay allowed for each cycle.
    """
    tolerance = [] # list of tolerances for each cycle
    n_cyc = len(cyc_start) # number of cycles

    if n_cyc == 0:
        return tolerance
    
    for i in range(n_cyc-1):
        len_cyc = len(cyc_profile[i])
        gap = cyc_start[i+1] - (cyc_start[i] + len_cyc)
        if gap > max_delay: # maximum tolerance is 6 hours
                tolerance.append(max_delay)
        else:
                tolerance.append(gap)
    # the last cycle can be delayed until the end of the day
    len_cyc = len(cyc_profile[-1])
    gap = duration - (cyc_start[-1] + len_cyc) 
    if gap > max_delay:
        tolerance.append(max_delay)
    else:
        tolerance.append(gap)
    
    return tolerance


load the houldhold data and the dynamic price

In [84]:
household = pd.read_csv('summer_household_energydata.csv', index_col=0)
duration = 24 * 4  # 24 hours, 15 minutes per time step per day
n_days = household.shape[0] // duration

# initial time series
critical_tot = np.array(household['Critical Appliances'])
wm_tot = np.array(household['Washing Machine'])
dishwasher_tot = np.array(household['Dishwasher'])
pv_tot = np.array(household['PV']) # The assumption is that our prediction of pv generation is 100% accurate
price = np.array(household['Dynamic Price']) / 1000 # convert to euro/kWh

# -------------------- settings -------------------- #
# pv
pv_battery_max = 16 # battery capacity in kWh
pv_battery_energy_rate = 1 # kWh
pv_battery_init = 0 # random
pv_battery_eff = 0.95 # efficiency of the battery charging/discharging process

# ev
home_charger = 4 # maximum power of home charger in kW
energy_rate = home_charger / 4 # maximum amout of energy that can be charged in 15 minutes
ev_in = 24 # plug in at 2016-05-01 18:00
ev_out = 88 # plug out at 2016-05-02 10:00

soc_max = 67 # ev battery capacity in kWh taken from the Tesla Model Y
soc_demand = 6 # energy demand of the EV in kWh, the assumption is that the EV is fully charged at the end of the day
soc_init = soc_max - soc_demand
ev_eff = 0.95 # efficiency of the EV charging/discharging process

# price
price_penalty = 0.1 # tuning parameter for the price penalty
power_th = 3 # threshold for peak load in kW


Benchmark

In [85]:
cost_benchmark = []

for day in range(n_days):
    wm_ini = wm_tot[day*duration:(day+1)*duration] # washing machine profile for the current day
    dishwasher_ini = dishwasher_tot[day*duration:(day+1)*duration] # dishwasher profile for the current day
    pv_ini = pv_tot[day*duration:(day+1)*duration] # pv profile for the current day
    critical_ini = critical_tot[day*duration:(day+1)*duration] # critical appliances profile for the current day
    ev_ini = np.zeros(duration) # initialize the EV profile
    ev_ini[ev_in:ev_in+int(np.ceil(soc_demand/energy_rate))] = energy_rate # dumb charging of EV, charging at 4 kW starting at 18:00 to fill the depletion of 6 kWh

    price_ini = price[day*duration:(day+1)*duration] # price profile for the current day
        
    tot_demand = dishwasher_ini + critical_ini + wm_ini + ev_ini
    cost_per_day = np.dot(tot_demand, price_ini) + price_penalty * np.sum(tot_demand[tot_demand > power_th/4]) # cost of the benchmark scenarioeuro')
    cost_benchmark.append(cost_per_day)
print('Cost benchmark: ', cost_benchmark, 'euro')

Cost benchmark:  [1.3856412599999999, 1.28336296, 1.0647579600000001, 1.1820179, 1.17091506, 1.39229425, 1.2430541800000001] euro


Scenario 1: unidirectional EV only

In [74]:
# Initialize the data structure in order to store the optimal profiles
data_scenario1 = pd.DataFrame(
    0,
    index=range(duration*n_days),  # 7 days in a week
    columns=['wm', 'dw', 'pv_bess', 'ev', 'critical']
)

cost_scenario1 = []

for day in range(n_days):
    print("days:", day)
    wm_ini = wm_tot[day*duration:(day+1)*duration] # washing machine profile for the current day
    dishwasher_ini = dishwasher_tot[day*duration:(day+1)*duration] # dishwasher profile for the current day
    pv_ini = pv_tot[day*duration:(day+1)*duration] # pv profile for the current day
    critical_ini = critical_tot[day*duration:(day+1)*duration] # critical appliances profile for the current day
    ev_ini = np.zeros(duration) # initialize the EV profile
    ev_ini[ev_in:ev_in+int(np.ceil(soc_demand/energy_rate))] = energy_rate # dumb charging of EV, charging at 4 kW starting at 18:00 to fill the depletion of 6 kWh

    price_ini = price[day*duration:(day+1)*duration] # price profile for the current day

    wm_cyc_start, wm_cyc_profile = extract_cyc(wm_ini) # start times and profiles of the cycles
    dw_cyc_start, dw_cyc_profile = extract_cyc(dishwasher_ini) # start times and profiles of the cycles

    # determin the tolerance for the washing machine and dishwasher cycles
    wm_tolerance = get_tolerance(wm_cyc_start, wm_cyc_profile, duration)
    dw_tolerance = get_tolerance(dw_cyc_start, dw_cyc_profile, duration)
    
    #########################  Scenario 1 #########################
    # Create a Gurobi model
    model1 = gp.Model(f"Scenario 1: uni EV, day{day+1}")

    # Washing machine related variables
    if wm_cyc_start:
        print(f"day {day}, wm_cyc_start:{wm_cyc_start}")
        wm_energy, _ = constrain_nonflex(model1, wm_cyc_start, wm_cyc_profile, duration, wm_tolerance, "wm")
        model1.update()

    if dw_cyc_start:
        # Dishwasher related variables
        dw_energy, _ = constrain_nonflex(model1, dw_cyc_start, dw_cyc_profile, duration, dw_tolerance, "dw")
        model1.update()

    # EV related variables
    ev_pos_energy, ev_neg_energy, ev_soc = constrain_flex(model1, soc_init, soc_demand, soc_max, energy_rate, 0, ev_eff, ev_in, ev_out, duration, "ev")
    model1.update()


    tot_demand = [] # add profiles of all cycles together to get the final wm power profile
    for t in range(duration):
        demand = gp.quicksum(wm_energy[i, t] for i in range(len(wm_cyc_start)))
        demand += gp.quicksum(dw_energy[i, t] for i in range(len(dw_cyc_start)))
        demand += ev_pos_energy[t] + ev_neg_energy[t] + critical_ini[t]
        model1.addConstr(demand >= 0, name=f"demand_nonneg_{t}") # ensure that the demand is non-negative

        tot_demand.append(demand) 

    # Now deal with the objective function considering the violation of power threshold
    excess = model1.addVars(duration,
                        lb=0,
                        vtype=gp.GRB.CONTINUOUS,
                        name="excess_kw")
    for t in range(duration):
        model1.addConstr(
            excess[t] >= tot_demand[t] - power_th/4,
            name=f"ex_def_{t}"
        )
        
    # binary flag: 1 if we exceed (excess>0), 0 otherwise
    penalty_flag = model1.addVars(duration,
                            vtype=gp.GRB.BINARY,
                            name="exceed_flag")

    # big‐M constraint: if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
    M = 1000.0   # “big‐M”: must exceed any possible (tot_demand*4 - P_th)
    for t in range(duration):
        # if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
        model1.addConstr(excess[t] <= M * penalty_flag[t],
                        name=f"flag_def_{t}")
        
    # cost part 1: cost of the energy consumed 
    # after you’ve set up penalty_flag[t] exactly as before…

    # Build a single “effective price” per t:
    cost = gp.quicksum(
        (
        (1 - penalty_flag[t]) * price[t] 
        +  penalty_flag[t] * (price_penalty)
        )
        * tot_demand[t]
        for t in range(duration)
    )

    # cost = gp.quicksum(tot_demand[t] * price[t] for t in range(duration))

    model1.setObjective(cost, gp.GRB.MINIMIZE)
    # Solve the model
    model1.optimize()
    
    
    # Check the optimization status
    if model1.status == gp.GRB.OPTIMAL:
        print("Optimal solution found.")
        print("Objective value (cost):", model1.objVal)

        cost_scenario1.append(model1.objVal)
        
        # Get the profiles of the appliances and store them in the data structure
        if wm_cyc_start:
            data_scenario1.loc[day*duration:(day+1)*duration-1, 'wm'] = np.array(get_profile(wm_energy, len(wm_cyc_start), duration, type='nonflex'))
        if dw_cyc_start:
            data_scenario1.loc[day*duration:(day+1)*duration-1, 'dw'] = np.array(get_profile(dw_energy, len(dw_cyc_start), duration, type='nonflex'))
        ev_pos_opt = get_profile(ev_pos_energy, duration, duration, type='flex')
        ev_neg_opt = get_profile(ev_neg_energy, duration, duration, type='flex')
        data_scenario1.loc[day*duration:(day+1)*duration-1, 'ev'] = np.array(ev_pos_opt) + np.array(ev_neg_opt)
        data_scenario1.loc[day*duration:(day+1)*duration-1, 'critical'] = critical_ini
    
    else:
        print("No optimal solution found. Status code:", model1.status)

    # print(day)
        

days: 0
day 0, wm_cyc_start:[79]
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 616 rows, 833 columns and 3337 nonzeros
Model fingerprint: 0xde972a07
Model has 384 quadratic objective terms
Model has 64 quadratic constraints
Variable types: 545 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e-04, 5e-02]
  QObjective range [9e-02, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [4e-03, 7e+01]
Presolve removed 514 rows and 638 columns
Presolve time: 0.00s
Presolved: 204 rows, 297 columns, 766 nonzeros
Variable types: 198 continuous, 99 integer (99 binary)
Found heuristic solution: objective 0.5468968
Found heuristic solution: objective 0.5460695

Root rela

In [75]:
print(cost_scenario1)
data_scenario1.to_csv('output/winter_scenario1.csv')

[0.5441954184210526, 0.6945130094736844, 0.5437344447368421, 0.5338991794736843, 0.9417905352631581, 0.6435442447368422, 0.5328811626315791]


Scenario 2: bidirectional EV

In [76]:
# Initialize the data structure in order to store the optimal profiles
data_scenario2 = pd.DataFrame(
    0,
    index=range(duration*n_days),  # 7 days in a week
    columns=['wm', 'dw', 'pv_bess', 'ev', 'critical']
)

cost_scenario2 = []

for day in range(n_days):
    print("days:", day)
    wm_ini = wm_tot[day*duration:(day+1)*duration] # washing machine profile for the current day
    dishwasher_ini = dishwasher_tot[day*duration:(day+1)*duration] # dishwasher profile for the current day
    pv_ini = pv_tot[day*duration:(day+1)*duration] # pv profile for the current day
    critical_ini = critical_tot[day*duration:(day+1)*duration] # critical appliances profile for the current day
    ev_ini = np.zeros(duration) # initialize the EV profile
    ev_ini[ev_in:ev_in+int(np.ceil(soc_demand/energy_rate))] = energy_rate # dumb charging of EV, charging at 4 kW starting at 18:00 to fill the depletion of 6 kWh

    price_ini = price[day*duration:(day+1)*duration] # price profile for the current day

    wm_cyc_start, wm_cyc_profile = extract_cyc(wm_ini) # start times and profiles of the cycles
    dw_cyc_start, dw_cyc_profile = extract_cyc(dishwasher_ini) # start times and profiles of the cycles

    # determin the tolerance for the washing machine and dishwasher cycles
    wm_tolerance = get_tolerance(wm_cyc_start, wm_cyc_profile, duration)
    dw_tolerance = get_tolerance(dw_cyc_start, dw_cyc_profile, duration)
    
    #########################  Scenario 1 #########################
    # Create a Gurobi model
    model2 = gp.Model(f"Scenario 2: bi EV, day{day+1}")

    # Washing machine related variables
    if wm_cyc_start:
        print(f"day {day}, wm_cyc_start:{wm_cyc_start}")
        wm_energy, _ = constrain_nonflex(model2, wm_cyc_start, wm_cyc_profile, duration, wm_tolerance, "wm")
        model2.update()

    if dw_cyc_start:
        # Dishwasher related variables
        dw_energy, _ = constrain_nonflex(model2, dw_cyc_start, dw_cyc_profile, duration, dw_tolerance, "dw")
        model2.update()

    # EV related variables
    ev_pos_energy, ev_neg_energy, ev_soc = constrain_flex(model2, soc_init, soc_demand, soc_max, energy_rate, -energy_rate, ev_eff, ev_in, ev_out, duration, "ev")
    model2.update()


    tot_demand = [] # add profiles of all cycles together to get the final wm power profile
    for t in range(duration):
        demand = gp.quicksum(wm_energy[i, t] for i in range(len(wm_cyc_start)))
        demand += gp.quicksum(dw_energy[i, t] for i in range(len(dw_cyc_start)))
        demand += ev_pos_energy[t] + ev_neg_energy[t] + critical_ini[t]
        model2.addConstr(demand >= 0, name=f"demand_nonneg_{t}") # ensure that the demand is non-negative
        tot_demand.append(demand) 

    # Now deal with the objective function considering the violation of power threshold
    excess = model2.addVars(duration,
                        lb=0,
                        vtype=gp.GRB.CONTINUOUS,
                        name="excess_kw")
    for t in range(duration):
        model2.addConstr(
            excess[t] >= tot_demand[t] - power_th/4,
            name=f"ex_def_{t}"
        )
        
    # binary flag: 1 if we exceed (excess>0), 0 otherwise
    penalty_flag = model2.addVars(duration,
                            vtype=gp.GRB.BINARY,
                            name="exceed_flag")

    # big‐M constraint: if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
    M = 1000.0   # “big‐M”: must exceed any possible (tot_demand*4 - P_th)
    for t in range(duration):
        # if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
        model2.addConstr(excess[t] <= M * penalty_flag[t],
                        name=f"flag_def_{t}")
        
    # cost part 1: cost of the energy consumed 
    # after you’ve set up penalty_flag[t] exactly as before…

    # Build a single “effective price” per t:
    cost = gp.quicksum(
        (
        (1 - penalty_flag[t]) * price[t] 
        +  penalty_flag[t] * (price_penalty)
        )
        * tot_demand[t]
        for t in range(duration)
    )

    # cost = gp.quicksum(tot_demand[t] * price[t] for t in range(duration))

    model2.setObjective(cost, gp.GRB.MINIMIZE)
    # Solve the model
    model2.optimize()
    
    
    # Check the optimization status
    if model2.status == gp.GRB.OPTIMAL:
        print("Optimal solution found.")
        print("Objective value (cost):", model2.objVal)

        cost_scenario2.append(model2.objVal)

        # Get the profiles of the appliances and store them in the data structure
        if wm_cyc_start:
            data_scenario2.loc[day*duration:(day+1)*duration-1, 'wm'] = np.array(get_profile(wm_energy, len(wm_cyc_start), duration, type='nonflex'))
        if dw_cyc_start:
            data_scenario2.loc[day*duration:(day+1)*duration-1, 'dw'] = np.array(get_profile(dw_energy, len(dw_cyc_start), duration, type='nonflex'))
        ev_pos_opt = get_profile(ev_pos_energy, duration, duration, type='flex')
        ev_neg_opt = get_profile(ev_neg_energy, duration, duration, type='flex')
        data_scenario2.loc[day*duration:(day+1)*duration-1, 'ev'] = np.array(ev_pos_opt) + np.array(ev_neg_opt)
        data_scenario2.loc[day*duration:(day+1)*duration-1, 'critical'] = critical_ini

    else:
        print("No optimal solution found. Status code:", model2.status)

    # print(day)
        

days: 0
day 0, wm_cyc_start:[79]
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 616 rows, 833 columns and 3337 nonzeros
Model fingerprint: 0xb3daf0b8
Model has 384 quadratic objective terms
Model has 64 quadratic constraints
Variable types: 545 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e-04, 5e-02]
  QObjective range [9e-02, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [4e-03, 7e+01]
Presolve removed 295 rows and 456 columns
Presolve time: 0.01s
Presolved: 486 rows, 542 columns, 1650 nonzeros
Variable types: 380 continuous, 162 integer (162 binary)
Found heuristic solution: objective 0.5351162
Found heuristic solution: objective 0.5336527

Root r

In [77]:
print(cost_scenario2)
data_scenario2.to_csv('output/winter_scenario2.csv', index=False)

[0.5079779736842104, 0.6481413411250002, 0.4959133155955685, 0.5002327252354578, 0.8313279752499994, 0.5974566099168981, 0.49220371005540164]


Scenario 3: PV + Battery, Dummy EV

In [78]:
# Initialize the data structure in order to store the optimal profiles
data_scenario3 = pd.DataFrame(
    0,
    index=range(duration*n_days),  # 7 days in a week
    columns=['wm', 'dw', 'pv_bess', 'ev', 'critical']
)

cost_scenario3 = []

for day in range(n_days):
    print("days:", day)
    wm_ini = wm_tot[day*duration:(day+1)*duration] # washing machine profile for the current day
    dishwasher_ini = dishwasher_tot[day*duration:(day+1)*duration] # dishwasher profile for the current day
    pv_ini = pv_tot[day*duration:(day+1)*duration] # pv profile for the current day
    critical_ini = critical_tot[day*duration:(day+1)*duration] # critical appliances profile for the current day
    ev_ini = np.zeros(duration) # initialize the EV profile
    ev_ini[ev_in:ev_in+int(np.ceil(soc_demand/energy_rate))] = energy_rate # dumb charging of EV, charging at 4 kW starting at 18:00 to fill the depletion of 6 kWh

    price_ini = price[day*duration:(day+1)*duration] # price profile for the current day

    wm_cyc_start, wm_cyc_profile = extract_cyc(wm_ini) # start times and profiles of the cycles
    dw_cyc_start, dw_cyc_profile = extract_cyc(dishwasher_ini) # start times and profiles of the cycles

    # determin the tolerance for the washing machine and dishwasher cycles
    wm_tolerance = get_tolerance(wm_cyc_start, wm_cyc_profile, duration)
    dw_tolerance = get_tolerance(dw_cyc_start, dw_cyc_profile, duration)
    
    #########################  Scenario 3 #########################
    # Create a Gurobi model
    model3 = gp.Model(f"Scenario 3: PV+BESS, day{day+1}")

    # Washing machine related variables
    if wm_cyc_start:
        wm_energy, _ = constrain_nonflex(model3, wm_cyc_start, wm_cyc_profile, duration, wm_tolerance, "wm")
        model3.update()

    if dw_cyc_start:
        # Dishwasher related variables
        dw_energy, _ = constrain_nonflex(model3, dw_cyc_start, dw_cyc_profile, duration, dw_tolerance, "dw")
        model3.update()

    # PV and BESS related variables
    battery_pos_energy, battery_neg_energy, _ = constrain_pv(model3, pv_battery_init, pv_battery_max, pv_battery_energy_rate, pv_battery_eff, duration, 'pv')
    model3.update()

    tot_demand = [] # add profiles of all cycles together to get the final wm power profile
    for t in range(duration):
        demand = gp.quicksum(wm_energy[i, t] for i in range(len(wm_cyc_start)))
        demand += gp.quicksum(dw_energy[i, t] for i in range(len(dw_cyc_start)))
        demand += critical_ini[t] + ev_ini[t]
        demand += battery_pos_energy[t] + battery_neg_energy[t] - pv_ini[t] # the energy demand is the sum of the washing machine, dishwasher, critical appliances, EV and the battery
        model3.addConstr(demand >= 0, name=f"demand_nonneg_{t}") # ensure that the demand is non-negative
        tot_demand.append(demand) 



    # Now deal with the objective function considering the violation of power threshold
    excess = model3.addVars(duration,
                        lb=0,
                        vtype=gp.GRB.CONTINUOUS,
                        name="excess_kw")
    for t in range(duration):
        model3.addConstr(
            excess[t] >= tot_demand[t] - power_th/4,
            name=f"ex_def_{t}"
        )
        
    # binary flag: 1 if we exceed (excess>0), 0 otherwise
    penalty_flag = model3.addVars(duration,
                            vtype=gp.GRB.BINARY,
                            name="exceed_flag")

    # big‐M constraint: if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
    M = 1000.0   # “big‐M”: must exceed any possible (tot_demand*4 - P_th)
    for t in range(duration):
        # if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
        model3.addConstr(excess[t] <= M * penalty_flag[t],
                        name=f"flag_def_{t}")
        
    cost = gp.quicksum(
        (
        (1 - penalty_flag[t]) * price[t] 
        +  penalty_flag[t] * (price_penalty)
        )
        * tot_demand[t]
        for t in range(duration)
    )
    model3.setObjective(cost, gp.GRB.MINIMIZE)
    # Solve the model
    model3.optimize()

    # Check the optimization status
    if model3.status == gp.GRB.OPTIMAL:
        print("Optimal solution found.")
        print("Objective value (cost):", model3.objVal)
        cost_scenario3.append(model3.objVal)

        # Get the profiles of the appliances and store them in the data structure
        if wm_cyc_start:
            data_scenario3.loc[day*duration:(day+1)*duration-1, 'wm'] = np.array(get_profile(wm_energy, len(wm_cyc_start), duration, type='nonflex'))
        if dw_cyc_start:
            data_scenario3.loc[day*duration:(day+1)*duration-1, 'dw'] = np.array(get_profile(dw_energy, len(dw_cyc_start), duration, type='nonflex'))
        bess_pos_opt = get_profile(battery_pos_energy, duration, duration, type='flex')
        bess_neg_opt = get_profile(battery_neg_energy, duration, duration, type='flex')
        data_scenario3.loc[day*duration:(day+1)*duration-1, 'pv_bess'] = np.array(bess_pos_opt) + np.array(bess_neg_opt)
        data_scenario3.loc[day*duration:(day+1)*duration-1, 'ev'] = ev_ini
        data_scenario3.loc[day*duration:(day+1)*duration-1, 'critical'] = critical_ini

    else:
        print("No optimal solution found. Status code:", model3.status)
    
        

days: 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 583 rows, 865 columns and 3400 nonzeros
Model fingerprint: 0x651d9181
Model has 384 quadratic objective terms
Model has 96 quadratic constraints
Variable types: 577 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-04, 1e-01]
  QObjective range [9e-02, 2e-01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [9e-03, 1e+00]
Presolve removed 166 rows and 366 columns
Presolve time: 0.01s
Presolved: 638 rows, 720 columns, 2115 nonzeros
Variable types: 509 continuous, 211 integer (211 binary)
Found heuristic solution: objective 0.4613087
Found heuristic solution: objective 0.4577564
Found heuristic solution: object

In [79]:
print(cost_scenario3)
data_scenario3.to_csv('output/winter_scenario3.csv', index=False)

[0.31994686531163374, 0.49426883507991637, 0.3015082011175892, 0.28074628318559475, 0.7343764213160666, 0.48402208759556764, 0.3166657966274232]


Benchmark: w/o PV, w/o demand response
meaning EV is charged using the dumb charging strategy. The penalty works in the following way: if the total load exceeds the threshold, the price will become the product of the maximum price and a tuning parameter.

Scenario 4: All together

In [80]:
# Initialize the data structure in order to store the optimal profiles
data_scenario4 = pd.DataFrame(
    0,
    index=range(duration*n_days),  # 7 days in a week
    columns=['wm', 'dw', 'pv_bess', 'ev', 'critical']
)

cost_scenario4 = []

for day in range(n_days):
    print("days:", day)
    wm_ini = wm_tot[day*duration:(day+1)*duration] # washing machine profile for the current day
    dishwasher_ini = dishwasher_tot[day*duration:(day+1)*duration] # dishwasher profile for the current day
    pv_ini = pv_tot[day*duration:(day+1)*duration] # pv profile for the current day
    critical_ini = critical_tot[day*duration:(day+1)*duration] # critical appliances profile for the current day
    ev_ini = np.zeros(duration) # initialize the EV profile
    ev_ini[ev_in:ev_in+int(np.ceil(soc_demand/energy_rate))] = energy_rate # dumb charging of EV, charging at 4 kW starting at 18:00 to fill the depletion of 6 kWh

    price_ini = price[day*duration:(day+1)*duration] # price profile for the current day

    wm_cyc_start, wm_cyc_profile = extract_cyc(wm_ini) # start times and profiles of the cycles
    dw_cyc_start, dw_cyc_profile = extract_cyc(dishwasher_ini) # start times and profiles of the cycles

    # determin the tolerance for the washing machine and dishwasher cycles
    wm_tolerance = get_tolerance(wm_cyc_start, wm_cyc_profile, duration)
    dw_tolerance = get_tolerance(dw_cyc_start, dw_cyc_profile, duration)
    
    #########################  Scenario 4 #########################
    # Create a Gurobi model
    model4 = gp.Model(f"Scenario 4: All together, day{day+1}")

    # Washing machine related variables
    if wm_cyc_start:
        wm_energy, _ = constrain_nonflex(model4, wm_cyc_start, wm_cyc_profile, duration, wm_tolerance, "wm")
        model4.update()

    if dw_cyc_start:
        # Dishwasher related variables
        dw_energy, _ = constrain_nonflex(model4, dw_cyc_start, dw_cyc_profile, duration, dw_tolerance, "dw")
        model4.update()

    # EV related variables
    ev_pos_energy, ev_neg_energy, _ = constrain_flex(model4, soc_init, soc_demand, soc_max, energy_rate, -energy_rate, ev_eff, ev_in, ev_out, duration, "ev")
    model4.update()

    # PV and BESS related variables
    battery_pos_energy, battery_neg_energy, _ = constrain_pv(model4, pv_battery_init, pv_battery_max, pv_battery_energy_rate, pv_battery_eff, duration, 'pv')
    model4.update()

    tot_demand = [] # add profiles of all cycles together to get the final wm power profile
    for t in range(duration):
        demand = gp.quicksum(wm_energy[i, t] for i in range(len(wm_cyc_start)))
        demand += gp.quicksum(dw_energy[i, t] for i in range(len(dw_cyc_start)))
        demand += ev_pos_energy[t] + ev_neg_energy[t] + critical_ini[t]
        demand += battery_pos_energy[t] + battery_neg_energy[t] - pv_ini[t] # the energy demand is the sum of the washing machine, dishwasher, critical appliances, EV and the battery
        model4.addConstr(demand >= 0, name=f"demand_nonneg_{t}") # ensure that the demand is non-negative
        tot_demand.append(demand) 

    # Now deal with the objective function considering the violation of power threshold
    excess = model4.addVars(duration,
                        lb=0,
                        vtype=gp.GRB.CONTINUOUS,
                        name="excess_kw")
    for t in range(duration):
        model4.addConstr(
            excess[t] >= tot_demand[t] - power_th/4,
            name=f"ex_def_{t}"
        )
        
    # binary flag: 1 if we exceed (excess>0), 0 otherwise
    penalty_flag = model4.addVars(duration,
                            vtype=gp.GRB.BINARY,
                            name="exceed_flag")

    # big‐M constraint: if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
    M = 1000.0   # “big‐M”: must exceed any possible (tot_demand*4 - P_th)
    for t in range(duration):
        # if excess[t]>0 ⇒ flag[t]=1, else flag[t] can be 0
        model4.addConstr(excess[t] <= M * penalty_flag[t],
                        name=f"flag_def_{t}")
        
    cost = gp.quicksum(
        (
        (1 - penalty_flag[t]) * price[t] 
        +  penalty_flag[t] * (price_penalty)
        )
        * tot_demand[t]
        for t in range(duration)
    )
    model4.setObjective(cost, gp.GRB.MINIMIZE)
    # Solve the model
    model4.optimize()

    # Check the optimization status
    if model4.status == gp.GRB.OPTIMAL:
        print("Optimal solution found.")
        print("Objective value (cost):", model4.objVal)
        cost_scenario4.append(model4.objVal)

        # Get the profiles of the appliances and store them in the data structure
        if wm_cyc_start:
            data_scenario4.loc[day*duration:(day+1)*duration-1, 'wm'] = np.array(get_profile(wm_energy, len(wm_cyc_start), duration, type='nonflex'))
        if dw_cyc_start:
            data_scenario4.loc[day*duration:(day+1)*duration-1, 'dw'] = np.array(get_profile(dw_energy, len(dw_cyc_start), duration, type='nonflex'))
        bess_pos_opt = get_profile(battery_pos_energy, duration, duration, type='flex')
        bess_neg_opt = get_profile(battery_neg_energy, duration, duration, type='flex')
        data_scenario4.loc[day*duration:(day+1)*duration-1, 'pv_bess'] = np.array(bess_pos_opt) + np.array(bess_neg_opt)

        ev_pos_opt = get_profile(ev_pos_energy, duration, duration, type='flex')
        ev_neg_opt = get_profile(ev_neg_energy, duration, duration, type='flex')
        data_scenario4.loc[day*duration:(day+1)*duration-1, 'ev'] = np.array(ev_pos_opt) + np.array(ev_neg_opt)
        data_scenario4.loc[day*duration:(day+1)*duration-1, 'critical'] = critical_ini

    else:
        print("No optimal solution found. Status code:", model4.status)

days: 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 713 rows, 1122 columns and 4106 nonzeros
Model fingerprint: 0x24b5e757
Model has 576 quadratic objective terms
Model has 160 quadratic constraints
Variable types: 834 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-04, 5e-02]
  QObjective range [9e-02, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [9e-03, 7e+01]
Presolve removed 84 rows and 377 columns
Presolve time: 0.01s
Presolved: 977 rows, 1093 columns, 3215 nonzeros
Variable types: 819 continuous, 274 integer (274 binary)
Found heuristic solution: objective 0.4801682

Root relaxation: objective -3.024303e+00, 763 iterations, 0.01 seconds (0.0

In [81]:
print(cost_scenario4)
data_scenario4.to_csv('output/winter_scenario4.csv', index=False)

[0.29967875545263056, 0.44726645182825453, 0.26295770147397524, 0.2708808631914269, 0.6107271015731297, 0.4371139930747928, 0.29253073295277]
